In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("\u2022 Using TensorFlow Version:", tf.__version__)
print("\u2022 Using TensorFlow Hub Version: ", hub.__version__)
print(
    "\u2022 GPU Device Found."
    if tf.test.is_gpu_available()
    else "\u2022 GPU Device Not Found. Running on CPU"
)

• Using TensorFlow Version: 2.14.0
• Using TensorFlow Hub Version:  0.15.0
• GPU Device Found.


In [ ]:
module_selection = ("mobilenet_v2", 224, 1280) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


In [ ]:
path = os.path.join(os.getcwd(), "drive", "MyDrive")  # comment this for local use
datapath = os.path.join(path, "Dataset")  # change path to os.getcwd()
train_path = os.path.join(datapath, "Train")
test_path = os.path.join(datapath, "Test")

In [ ]:
# Define data augmentation transformations
train_datagen = ImageDataGenerator(
    rescale=1.0 / 127.5,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

Found 1275 files belonging to 17 classes.
Found 425 files belonging to 17 classes.


In [ ]:
# No data augmentation for validation set
test_datagen = ImageDataGenerator(rescale=1.0 / 127.5)

In [ ]:
# Create data generators
BATCH_SIZE = 32  # @param {type:"integer"}
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'  # Use 'sparse' for SparseCategoricalCrossentropy
)

validation_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'  # Use 'sparse' for SparseCategoricalCrossentropy
)

In [ ]:
do_fine_tuning = False  # @param {type:"boolean"}

In [ ]:
feature_extractor = hub.KerasLayer(
    MODULE_HANDLE,
    input_shape=IMAGE_SIZE + (3,),
    output_shape=[FV_SIZE],
    trainable=do_fine_tuning,
)

In [ ]:
print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential(
    [feature_extractor, tf.keras.layers.Dense(17, activation="softmax")]
)

model.summary()

In [ ]:
# @title (Optional) Unfreeze some layers
NUM_LAYERS = 17  # @param {type:"slider", min:1, max:50, step:1}

if do_fine_tuning:
    feature_extractor.trainable = True

    for layer in model.layers[-NUM_LAYERS:]:
        layer.trainable = True

else:
    feature_extractor.trainable = False

In [ ]:
if do_fine_tuning:
    model.compile(
        optimizer=tf.keras.optimizers.SGD(lr=0.002, momentum=0.9),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )
else:
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )

Building model with https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 17)                21777     
                                                                 
Total params: 2279761 (8.70 MB)
Trainable params: 21777 (85.07 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [ ]:
EPOCHS = 20
hist = model.fit(train_generator, epochs=EPOCHS, validation_data=validation_generator)

In [ ]:
RPS_SAVED_MODEL = "rps_saved_model"

In [ ]:
tf.saved_model.save(model, RPS_SAVED_MODEL)

In [ ]:
%%bash -s $RPS_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

In [ ]:
loaded = tf.saved_model.load(RPS_SAVED_MODEL)

In [ ]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(RPS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

In [ ]:
tflite_model_file = "converted_model.tflite"

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

In [ ]:
import shutil

shutil.make_archive("rps_saved_model", "zip", "rps_saved_model")